In [ ]:
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib import pylab
import datetime as dt
import pandas as pd
import calendar
%matplotlib inline

#13년도
csv_path13 = [0]*12
read13 = [0]*12
df13 = [0]*12
temp=""

#13년도
for i in range(2):
    #csv경로
    csv_path13[i] = "dataset\\일자별 위탁판매현황(2013년"+str(i+1)+"월).csv"
    #csv읽어옴
    read13[i] = pd.read_csv(csv_path13[i], engine='python',error_bad_lines=False, warn_bad_lines=False)
    #데이터프레임 생성
    df13[i] = DataFrame(read13[i])
    if i==1:
        temp = pd.concat([df13[i-1],df13[i]])
    elif i==0:
        df = df13[0]
    elif i==11:
        df = pd.concat([temp, df13[i]])
    elif i>1:
        temp = pd.concat([temp,df13[i]])

#어종상태명이 '(활)' 인 row의 해당 속성 열을 가져온다.
df = df.loc[df.어종상태명=="(활)",["위판일자","수산물표준코드명","산지조합명","위판중량", "위판금액"]]
#코드명이 '가자미류'인 row만 선택한다.
df = df.loc[df.수산물표준코드명=="가자미류",["위판일자","산지조합명","위판중량", "위판금액"]]
#"kg평균"의 비어있는 column을 추가한다.(여기에 (위판중량/위판금액)를 저장한다.)
df["kg평균"] = ""
for i in range(len(df)):
    kg = df.loc[i,["위판중량"]]
    money = df.loc[i,["위판금액"]]
    print(kg)
    print(money)
print(df)

'''
#그룹별 평균을 구한다. 여기서는 위판일자 별 위판단가 평균
df_mean = df.groupby(['위판일자','수산물표준코드명'], as_index=False).mean() 

#그래프 월별 색깔 지정 (10가지 색상)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

#한글 폰트 지정
plt.rcParams["font.family"] = 'NanumSquareRound'
plt.rcParams["font.size"] = 10

#x,y좌표 라벨, 표 크기 지정
plt.figure(figsize=(24,10))
plt.xlabel('날짜')
plt.ylabel('가격(원)')

#[plot data] x좌표 : 날짜, y좌표 : 일 평균 가격
x,y = [],[]
for line in df_mean.위판일자:
    times = dt.datetime.strptime(str(line),'%Y%m%d')
    x.append(times)
for line in df_mean.위판단가:
    y.append(line)

#강조 구간
for i in range(1,13):
    span_start = dt.datetime(2013, i, 1)
    end_day = calendar.monthrange(2013, i)[1] # i월달의 마지막 일을 가져온다.
    span_end = dt.datetime(2013, i, end_day)
    plt.axvspan(span_start, span_end, facecolor=colors[i%10], alpha=0.5)

plt.title("가자미류 13년도 일 평균 가격")
plt.plot(x,y,linestyle='-',c="m" ,label="value")
plt.show()
'''